## Consume data from Kafka stream

* in the previous ntb you published some data to Kafka stream
* read and process the data from the stream

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from pyspark.sql.types import *

spark = (
    SparkSession
    .builder
    .appName('kafka_test')
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0-preview")
    .getOrCreate()
)

In [ ]:
spark.version

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

checkpoint_location = os.path.join(project_path, 'output/streaming-output/checkpoint/4')

In [ ]:
stream_schema = StructType(
    [
        StructField('question_id', LongType()),
        StructField('creation_date', TimestampType()),
        StructField('title', StringType()),
        StructField('r', IntegerType())
    ]
)

In [ ]:
streaming_query = (
    spark
    .readStream
    .format('kafka')
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "myTop")
    .option('startingOffsets', 'earliest')
    .load()
)

In [ ]:
parsed_data = (
    streaming_query
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
)

In [ ]:
q = (
    parsed_data
    .writeStream
    .format('memory')
    .outputMode('append')
    .queryName('my_stream')
    .start()
)

In [ ]:
spark.sql("select * from my_stream").show()

In [ ]:
q.stop()

In [ ]:
spark.stop()